In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import json

# Census API Key
from config import (api_key, gkey)
c = Census(api_key, year=2016)
api_key

# gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
#load police killing data
file_path = "../police_killings.csv"

police_df = pd.read_csv(file_path)
police_df = police_df.rename(columns={ "Victim's race": "race",
                                     "Zipcode": "zip"})

police_df = police_df.replace({'Unknown race': 'Unknown',
                              "Unknown Race" : "Unknown"})

In [3]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E","B02001_002E", "B02001_003E", "B02001_004E", "B02001_005E", "B02001_006E", "B02001_008E", "B03001_003E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)
#rename columns 
census_pd = census_pd.rename(columns={ "B01003_001E": "population",
                                      "B02001_002E": "white",
                                      "B02001_003E": "black",
                                      "B02001_004E" : "native",
                                      "B02001_005E" : "asian",
                                      "B02001_006E" : "native_h",
                                      "B02001_008E" : "mixed",
                                      "B03001_003E" : "hispanic",                                      
                                      "zip code tabulation area": "Zipcode"})

In [4]:
#get percentages

census_pd["p_white"] = census_pd.white/census_pd.population
census_pd["p_black"] = census_pd.black/census_pd.population
census_pd["p_native"] = census_pd.native/census_pd.population
census_pd["p_asian"] = census_pd.asian/census_pd.population
census_pd["p_native_h"] = census_pd.native_h/census_pd.population
census_pd["p_mixed"] = census_pd.mixed/census_pd.population
census_pd["p_hispanic"] = census_pd.hispanic/census_pd.population


In [5]:
kill_df = police_df.groupby(["zip", "race"]).size()
kill_df = pd.DataFrame(kill_df).reset_index()
total_df = kill_df.pivot_table(columns="race", index='zip', values=0, fill_value=0)

In [6]:
#rename columns 
total_df = total_df.rename(columns={"White": "white_d",
                                      "Black": "black_d",
                                      "Native American" : "native_d",
                                      "Asian" : "asian_d",
                                      "Pacific Islander" : "native_h_d",
                                      "Hispanic" : "hispanic_d",                                      
                                      "zip code tabulation area": "Zipcode"})
#total column
total_df['total'] = total_df.asian_d + total_df.black_d + total_df.hispanic_d + total_df.native_d + total_df.Unknown + total_df.white_d
total_df

#percentages 
total_df['p_white_d'] = total_df.white_d/total_df.total
total_df['p_black_d'] = total_df.black_d/total_df.total
total_df['p_hispanic_d'] = total_df.hispanic_d/total_df.total
total_df['p_native_d'] = total_df.native_d/total_df.total
total_df['p_native_h_d'] = total_df.native_h_d/total_df.total
total_df['p_Unknown_d'] = total_df.Unknown/total_df.total
total_df['p_asian_d'] = total_df.asian_d/total_df.total

In [7]:
#join the two
total_df1 = total_df.reset_index()
census_pd = census_pd.rename(columns = {"Zipcode":"zip"})

#making type standard
total_df1['zip']=total_df1['zip'].astype(int)
census_pd['zip']=census_pd['zip'].astype(int)

everything = pd.merge(total_df1, census_pd, how = 'left', on=['zip', 'zip'])

In [8]:
#adding proportionalities
everything['white_popp'] = everything.white_d/everything.white
everything['black_popp'] = everything.black_d/everything.black
everything['hispanic_popp'] = everything.hispanic_d/everything.hispanic
everything['native_popp'] = everything.native_d/everything.native
everything['asian_popp'] = everything.asian_d/everything.asian

In [9]:
#adding population comparison data
everything['white_prop'] = everything.p_white_d/everything.p_white
everything['black_prop'] = everything.p_black_d/everything.p_black
everything['hispanic_prop'] = everything.p_hispanic_d/everything.p_hispanic
everything['native_perc_prop'] = everything.p_native_d/everything.p_native
everything['asian_perc_prop'] = everything.p_asian_d/everything.p_asian

In [10]:
everything.to_csv('everything.csv')
everything.columns

Index(['zip', 'asian_d', 'black_d', 'hispanic_d', 'native_d', 'native_h_d',
       'Unknown', 'white_d', 'total', 'p_white_d', 'p_black_d', 'p_hispanic_d',
       'p_native_d', 'p_native_h_d', 'p_Unknown_d', 'p_asian_d', 'population',
       'white', 'black', 'native', 'asian', 'native_h', 'mixed', 'hispanic',
       'p_white', 'p_black', 'p_native', 'p_asian', 'p_native_h', 'p_mixed',
       'p_hispanic', 'white_popp', 'black_popp', 'hispanic_popp',
       'native_popp', 'asian_popp', 'white_prop', 'black_prop',
       'hispanic_prop', 'native_perc_prop', 'asian_perc_prop'],
      dtype='object')

In [11]:
summary = everything[['zip', 'asian_d', 'black_d', 'hispanic_d', 'native_d', 'native_h_d',
       'Unknown', 'white_d', 'total', 'p_white_d', 'p_black_d', 'p_hispanic_d',
       'p_native_d', 'p_native_h_d', 'p_Unknown_d', 'p_asian_d', 'population',
       'white', 'black', 'native', 'asian', 'native_h', 'mixed', 'hispanic',
       'p_white', 'p_black', 'p_native', 'p_asian', 'p_native_h', 'p_mixed',
       'p_hispanic', 'white_popp', 'black_popp', 'hispanic_popp',
       'native_popp', 'asian_popp', 'white_prop', 'black_prop',
       'hispanic_prop', 'native_perc_prop', 'asian_perc_prop']]
summary

summary = summary.replace({np.inf : 100})

In [12]:
import folium

In [29]:
m = folium.Map(location=[48, -102], zoom_start=3)

url = 'https://data.ct.gov/api/views/2fi9-sgi3/rows.json?accessType=DOWNLOAD'
#zip_JSON = requests.get(url).json()

folium.Choropleth(
    geo_data=url,
    name='choropleth',
    data=summary,
    columns=['zip', 'black_popp'],
    key_on='feature.id'
    #fill_color='YlGn',
    #fill_opacity=0.7,
    #line_opacity=0.2,
    #legend_name='Black Death %'
).add_to(m)

folium.LayerControl().add_to(m)

m

KeyError: 'type'

In [28]:
zip_JSON['meta'].feature

AttributeError: 'dict' object has no attribute 'feature'